# Regression example: *Synthetic* 3 dataset
$$y  = (5x_1-5x_2)1_{x_5\le 0}+(5x_3-5x_4)1_{x_5> 0}$$



## Librairies 

In [1]:
import os, sys, time
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
import torch 
import torch.nn as nn
import torch.optim as optim 

In [4]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from skorch.callbacks import EarlyStopping,LRScheduler,Checkpoint, TrainEndCheckpoint, EpochScoring

In [5]:
from tabsra.skorch_tabsra import InputShapeSetterTabSRA,TabSRALinearRegressor

## Data

In [6]:
np.random.seed(42)
x1 = np.random.normal(0,1,3*10000)
x2 = np.random.normal(0,1,3*10000)
x3 = np.random.normal(0,1,3*10000)
x4 = np.random.normal(0,1,3*10000)
x5 = np.random.normal(0,1,3*10000)
X = np.concatenate((x1.reshape(-1,1), x2.reshape(-1,1),x3.reshape(-1,1),x4.reshape(-1,1),x5.reshape(-1,1)  ),axis=1)

In [7]:
odd = (5*x1-5*x2)*(x5<=0) + (5*x3-5*x4)*(x5>0)
y_c = 1/(1+np.exp(-odd))
y = np.where(y_c>0.5, 1,0)
X_train_,X_test_,Y_train_,Y_test_ = train_test_split(X,odd,stratify =y , random_state=42)
n_features, n_classes = 5,1
len(Y_train_),len(Y_test_)

(22500, 7500)

In [8]:
feature_names = ['x1','x2','x3','x4','x5']


## Model: TabSRALinear 

In [9]:
#valid_loss
other_params = {"module__encoder_bias":True,
                "module__classifier_bias":False,
                "module__n_head":1,
                "module__dim_head":4,
                "optimizer__lr":0.01,
                "max_epochs":150,
                "batch_size":512,                
                "optimizer__weight_decay":0,
                "random_state":42,
               }
scoring = EpochScoring(scoring='r2',lower_is_better=False)
setter = InputShapeSetterTabSRA(regression=True)
early_stop = EarlyStopping(monitor=scoring.scoring, patience=20,load_best=True,lower_is_better=False, threshold=0.00001,threshold_mode='abs')
lr_scheduler = LRScheduler(policy=ReduceLROnPlateau, patience=15, min_lr=2e-5,factor=0.2, verbose=1, mode='max',monitor=scoring.scoring)
call_b = [scoring, setter, early_stop, lr_scheduler]

In [10]:
%%time
TabSRA = TabSRALinearRegressor(**other_params,callbacks=call_b)
_ = TabSRA.fit(X_train_.astype(np.float32),Y_train_.reshape(-1,1).astype(np.float32))

Re-initializing module because the following parameters were re-set: module__dim_input, module__dim_output.
Re-initializing criterion.
Re-initializing optimizer.
  epoch      r2    train_loss    valid_loss      dur
-------  ------  ------------  ------------  -------
      1  0.1471       47.6918       41.5221  13.6775
      2  0.3002       39.1473       34.0702  0.2457
      3  0.4785       30.9727       25.3871  0.2001
      4  0.5967       23.2003       19.6363  0.2089
      5  0.6703       18.7371       16.0495  0.2016
      6  0.7209       15.5881       13.5877  0.2521
      7  0.7612       13.3134       11.6240  0.2149
      8  0.7855       11.6417       10.4425  0.1948
      9  0.8088       10.4440        9.3066  0.1913
     10  0.8264        9.4632        8.4532  0.1960
     11  0.8387        8.7079        7.8509  0.1996
     12  0.8467        8.2203        7.4637  0.1980
     13  0.8513        7.8352        7.2410  0.1948
     14  0.8577        7.6288        6.9299  0.1935
   

In [22]:
Coef  = pd.DataFrame(TabSRA.get_weights()[0])
Coef.columns=feature_names
Coef

,x1,x2,x3,x4,x5
0,5.001876,-4.996838,5.169649,-5.016213,5.380935


In [25]:
pred_tabsra = TabSRA.predict(X_test_.astype(np.float32))

rmse_tabsra = np.sqrt(mean_squared_error(Y_test_, pred_tabsra))
r2_tabsra = r2_score(Y_test_, pred_tabsra) 
print(f"--RMSE  = {rmse_tabsra} -- r2  = {r2_tabsra}")

--RMSE  = 0.3034210188095153 -- r2  = 0.9981866874934777


In [28]:
%%time
attributions_tabsra = pd.DataFrame(TabSRA.get_feature_attribution(X_test_.astype(np.float32)))
attributions_tabsra.columns = feature_names
attributions_tabsra['pred_proba'] = pred_tabsra
attributions_tabsra['label'] = Y_test_

CPU times: user 28.5 ms, sys: 0 ns, total: 28.5 ms
Wall time: 5.43 ms


In [29]:
attributions_tabsra.head()

,x1,x2,x3,x4,x5,pred_proba,label
0,-7.768471e-06,0.027804,8.204901e+00,-3.011704e+00,0.009502,5.230494,5.207175
1,2.296751e+00,1.677891,-4.941949e-08,-1.649862e-06,-0.000148,3.974492,3.974853
2,1.641793e+00,-0.204040,-6.330068e-09,1.415404e-09,-0.000368,1.437385,1.437009
3,-4.454783e+00,-9.539963,-5.557716e-05,-9.707033e-04,-0.003960,-13.999731,-14.000517
4,-9.808094e-07,-0.013421,2.677278e-01,5.371353e-01,0.044899,0.836339,0.807119


In [31]:
%%time
attention_tabsra = pd.DataFrame(TabSRA.get_attention(X_test_.astype(np.float32))[0])
attention_tabsra.columns = feature_names
attention_tabsra['pred_proba'] = pred_tabsra
attention_tabsra['label'] = Y_test_

CPU times: user 33.2 ms, sys: 0 ns, total: 33.2 ms
Wall time: 7.1 ms


In [32]:
attention_tabsra.head()

,x1,x2,x3,x4,x5,pred_proba,label
0,2.521361e-06,0.006143,9.666835e-01,9.999999e-01,0.002266,5.230494,5.207175
1,9.999952e-01,1.000000,2.758221e-08,2.596590e-07,0.000016,3.974492,3.974853
2,9.999999e-01,1.000000,4.321437e-10,4.146849e-09,0.000046,1.437385,1.437009
3,9.997753e-01,0.999958,8.022116e-04,2.500543e-04,0.001725,-13.999731,-14.000517
4,2.277387e-07,0.011819,9.529745e-01,1.000000e+00,0.008292,0.836339,0.807119
